<a href="https://colab.research.google.com/github/Dinusha-Ariyarathna/Angular-Services-DipendencyInjection/blob/main/TreasurehuntURL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install fastapi uvicorn[standard] pillow ultralytics pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 40.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [ultralytics]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from ultralytics import YOLO
from PIL import Image
import traceback, io, base64, os, uuid
from datetime import datetime

app = FastAPI()
model = YOLO("/content/drive/MyDrive/OD_Model/best.pt")

# Ensure the folder exists
SAVE_DIR = "received"
os.makedirs(SAVE_DIR, exist_ok=True)

class DetectRequest(BaseModel):
    image_data: str

@app.post("/detect")
def detect(req: DetectRequest):
    try:
        # 1) Decode base64 payload
        img_bytes = base64.b64decode(req.image_data)

        # 2) Save to disk with a timestamp + UUID
        timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
        filename  = f"{timestamp}_{uuid.uuid4().hex}.jpg"
        filepath  = os.path.join(SAVE_DIR, filename)
        with open(filepath, "wb") as f:
            f.write(img_bytes)

        # 3) Open as PIL image for inference
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

        # 4) Run your YOLO predict as before
        results = model.predict(source=img, conf=0.5, stream=False)

        # 5) Extract detections
        dets = []
        for r in results:
            for box in r.boxes:
                dets.append({
                    "name":       model.names[int(box.cls[0])],
                    "confidence": float(box.conf[0])
                })
        dets.sort(key=lambda x: -x["confidence"])
        return {"top": dets[:10]}

    except Exception as e:
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=str(e))



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
!pip install uvicorn
!cp /content/drive/MyDrive/OD_Model/best.pt /content/
!cp /content/drive/MyDrive/OD_Model/main.py /content/
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &> uvicorn.log &
!lsof -iTCP:8000 -sTCP:LISTEN

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2yUfHZuAM9mDuM1zmn7ZD7CZmXx_39gDc8nxxoCwzboLWXwhT")

In [ ]:
from pyngrok import ngrok

# Create a public URL to your local server port
public_url = ngrok.connect(8000, domain="resoluble-carlyn-sublustrous.ngrok-free.app")
print("👉 Your API is live at:", public_url)


👉 Your API is live at: NgrokTunnel: "https://resoluble-carlyn-sublustrous.ngrok-free.app" -> "http://localhost:8000"
